## Cell speed

In [1]:
%load_ext Cython

In [12]:
def pyadd(a, b):
    return a + b


In [9]:
%%cython

def cyadd(int a, int b):
    return a + b


In [10]:
%%timeit
pyadd(2,2)

95.7 ns ± 5.38 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [11]:
%%timeit
cyadd(2,2)

81.5 ns ± 14.5 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## A practical example

In [24]:
from PyQt5 import QtCore


class TypingInputHandler:
    hit_color = "green"
    miss_color = "red"

    def refresh(self, lesson_name=None, target_text=None):
        self.lesson_name = lesson_name
        self.hits = self.miss = 0
        self.accuracy = 0
        self.entered_text = ""
        self.target_text = target_text
        self.char_comparison_list = []
        self.finished = False
        if target_text is not None:
            self.display_text = '<span style="color:{}">{}</span>'.format(
                self.hit_color, self.target_text
            )
        else:
            self.display_text = ""

    def process_key_press(self, event):
        target_text = self.target_text
        if target_text is None:
            return
        event_key = event.key()

        if event_key == QtCore.Qt.Key_Backspace:
            if len(self.entered_text) > 0:
                self.entered_text = self.entered_text[:-1]
                self.char_comparison_list = self.char_comparison_list[:-1]
        else:
            char_entered = event.text()
            if len(char_entered) == 0:
                return
            self.entered_text += char_entered
            len_entered = len(self.entered_text)
            char_target = target_text[len_entered-1]
            if char_entered == char_target:
                color = self.hit_color
                self.hits += 1
            else:
                color = self.miss_color
                self.miss += 1
                # update our records for this typing error
                typing_error = TypingError(
                    self.lesson_name, char_entered, char_target
                )
                task = SaveTypingErrorsTask(typing_error)
                thread_pool.start(task)
                # replace red (incorrect) spaces with red asterix
                if char_target == " ":
                    char_target = "*"
            char_text = '<span style="color:{}">{}</span>'.format(
                color, char_target
            )

            self.accuracy = int(self.hits / (self.hits + self.miss) * 100)
            self.char_comparison_list.append(char_text)

        entered_text = "".join(self.char_comparison_list)
        remaining_text = target_text[len(self.entered_text):]
        self.display_text = entered_text + remaining_text
        if len(self.entered_text) >= len(target_text):
            self.finished = True


In [38]:
from collections import namedtuple


class Event:
    def __init__(self, key, text):
        self._key, self._text = key, text

    def key(self):
        return self._key

    def text(self):
        return self._text


event = Event(None, "B")
event

In [39]:
handler = TypingInputHandler()
handler

In [42]:
%%timeit
handler.refresh(lesson_name="Blah", target_text="Blah blah blah blah")
for char in handler.target_text:
    event = Event(None, char)
    handler.process_key_press(event)


57.5 µs ± 3.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [41]:
handler.finished

True

In [ ]:
%%cython
from PyQt5 import QtCore


class TypingInputHandler:
    hit_color = "green"
    miss_color = "red"

    def refresh(self, lesson_name=None, target_text=None):
        self.lesson_name = lesson_name
        self.hits = self.miss = 0
        self.accuracy = 0
        self.entered_text = ""
        self.target_text = target_text
        self.char_comparison_list = []
        self.finished = False
        if target_text is not None:
            self.display_text = '<span style="color:{}">{}</span>'.format(
                self.hit_color, self.target_text
            )
        else:
            self.display_text = ""

    def process_key_press(self, event):
        target_text = self.target_text
        if target_text is None:
            return
        event_key = event.key()

        if event_key == QtCore.Qt.Key_Backspace:
            if len(self.entered_text) > 0:
                self.entered_text = self.entered_text[:-1]
                self.char_comparison_list = self.char_comparison_list[:-1]
        else:
            cdef char* char_entered = event.text()
            cdef char* char_target = target_text[len_entered-1]
            if len(char_entered) == 0:
                return
            self.entered_text += char_entered
            len_entered = len(self.entered_text)
            if char_entered == char_target:
                color = self.hit_color
                self.hits += 1
            else:
                color = self.miss_color
                self.miss += 1
                # update our records for this typing error
                typing_error = TypingError(
                    self.lesson_name, char_entered, char_target
                )
                task = SaveTypingErrorsTask(typing_error)
                thread_pool.start(task)
                # replace red (incorrect) spaces with red asterix
                if char_target == " ":
                    char_target = "*"
            char_text = '<span style="color:{}">{}</span>'.format(
                color, char_target
            )

            self.accuracy = int(self.hits / (self.hits + self.miss) * 100)
            self.char_comparison_list.append(char_text)

        entered_text = "".join(self.char_comparison_list)
        remaining_text = target_text[len(self.entered_text):]
        self.display_text = entered_text + remaining_text
        if len(self.entered_text) >= len(target_text):
            self.finished = True